#### 【 지도학습 - 선형 모델: 다항회귀 】

- 주__제 : 농어의 길이에 따른 무게 예측 모델 구현
- 데이터 : ../Data/fish.csv
- 종__류 : 지도학습 - 수치값 예측 즉, 회귀
- 방__법 : 선형모델을 기반으로 구현 => LinearRegression

[1] 모듈 로딩 및 데이터 준비 <hr>

[1-1] 모듈 로딩

In [13]:
## [1-1] 모듈 로딩
## 기본 모듈
import numpy as np
import pandas as pd

## 시각화 모듈
import matplotlib.pyplot as plt
import koreanize_matplotlib

## ML 전처리 관련
from sklearn.preprocessing import PolynomialFeatures


## ML 학습 알고리즘 관련
from sklearn.linear_model import LinearRegression

## ML 성능평가 관련
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.metrics import mean_squared_error, root_mean_squared_error

##### [1-2] 실습 데이터 준비

In [14]:
featureNP = np.arange(4).reshape(2,2)
targetNP  = 1 + 2*featureNP[:, 0] + 3*featureNP[:, 0]**2 + 4*featureNP[:, 1]**3

print( featureNP.shape, featureNP.ndim )
print( featureNP )
print( targetNP )

(2, 2) 2
[[0 1]
 [2 3]]
[  5 125]


##### [2] 데이터 전처리 <HR>

In [15]:
## => 특성/피쳐 공학 => 피쳐 생성
## [기본] 2차 다항식 + 절편 사용 + 제곱값 사용
poly = PolynomialFeatures()
feature2 = poly.fit_transform(featureNP)
feature2

array([[1., 0., 1., 0., 0., 1.],
       [1., 2., 3., 4., 6., 9.]])

In [23]:
## [설정] 2차 다항식 + 절편 사용 + 제곱값 미사용
poly = PolynomialFeatures(interaction_only=True)
feature2 = poly.fit_transform(featureNP)
feature2

array([[1., 0., 1., 0.],
       [1., 2., 3., 6.]])

In [29]:
## [설정] 2차 다항식 + 절편 미사용 + 제곱값 미사용
poly = PolynomialFeatures(interaction_only=True, include_bias=False)
feature2 = poly.fit_transform(featureNP)
feature2

array([[0., 1., 0.],
       [2., 3., 6.]])

In [28]:
## [설정] 3차 다항식 + 절편 미사용 + 제곱값 미사용
poly = PolynomialFeatures(degree=3)
feature3 = poly.fit_transform(featureNP)
feature3

array([[ 1.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  1.],
       [ 1.,  2.,  3.,  4.,  6.,  9.,  8., 12., 18., 27.]])

##### [3] 학습 진행 <hr>

In [26]:
## 2차 다항식 피쳐
lrModel = LinearRegression()
lrModel.fit(feature2, targetNP)

print(f'피쳐별 가중치 : {lrModel.coef_}')
print(f'피쳐별 가중치 : {lrModel.intercept_}')

피쳐별 가중치 : [ 5.45454545  5.45454545 16.36363636]
피쳐별 가중치 : -0.45454545454545325


In [ ]:
## 3차 다항식 피쳐
lrModel = LinearRegression()
lrModel.fit(feature3, targetNP)
score = lrModel.score(feature3, targetNP)  ## r2_score

print(f'피쳐별 가중치 : {lrModel.coef_}')
print(f'피쳐별 가중치 : {lrModel.intercept_}')ex01_fish_poly
print(f'R2_score    : {score}')